# MATH3090 Assignment 1

Student: Hugo Burton (s4698512)
Date Due: Tuesday March 19 @ 1pm

In [51]:
import math
from typing import Dict
from colorama import Fore
import numpy as np
import sys

import bond
import interest
import newtons
import display

## Question 1 (6 marks)

### Part A (3 marks)


Suppose a company issues a zero coupon bond with face value $10,000 and which matures in 20 years. Calculate the price given:

In [52]:
face_value = 10_000
years_to_maturity = 20

i. An 8\% discount compound annual yield, compounded annually.

In [53]:
# Question i
interest_rate = 0.08
compounding_frequency_yr = 1

price_i = bond.price_zero_coupon_bond_discrete(
    face_value, years_to_maturity, interest_rate, compounding_frequency_yr)

display.display_answer(price_i)

Answer:$2145.48
----------------------------------------------------------------------------------------------------


ii. An 8\% discount continuous annual yield, compounded semi-annually.

In [54]:
# Question ii
interest_rate = 0.08

price_ii = bond.price_zero_coupon_bond_continuous(
    face_value, years_to_maturity, interest_rate)

display.display_answer(price_ii)

Answer:$2018.97
----------------------------------------------------------------------------------------------------


iii. A nonconstant yield of $y(t) = 0.06 + 0.2 t e^{-t^2}$.

In [55]:
# Question iii
q = "iii"
def yield_function(t): return 0.06 + 0.2 * t * math.exp(-t**2)

price_iii = bond.price_zero_coupon_bond_nonconstant_yield(
    face_value, years_to_maturity, yield_function)

display.display_answer(price_iii)

Answer:$2725.32
----------------------------------------------------------------------------------------------------


### Part B (3 marks)

A 10 year \$10,000 government bond has a coupon rate of 5\% payable quarterly and yields 7\%. Calculate the price.

In [56]:
# Part b (3 marks)
face_value = 10_000
years_to_maturity = 10
coupon_rate = 0.05
interest_rate = 0.07
compounding_frequency_yr = 4

price_b = bond.price_coupon_bearing_bond_discrete(
    face_value, years_to_maturity, coupon_rate, interest_rate, compounding_frequency_yr)

display.display_answer(price_b)

Answer:$8570.29
----------------------------------------------------------------------------------------------------


## Question 2 (6 marks)

Consider the cash flow 

$$C_0 = -3x,\hspace{0.5cm} C_1 = 5,\hspace{0.5cm} C_2 = x$$

(at periods 0, 1, 2 respectively) for some $x > 0$.

## Part A (3 marks)

Apply the discount process $d(k) = (1 + r)^{-k}$ so that the present value is

$$P = \sum_{k=0}^{2} d(k) C_k$$

What is the range of $x$ such that $P > 0$ when $r = 5\%$?

\begin{align}
    0 &< P \\
    0 &< \sum_{i=0}^{2} d(k) C_k \\
    0 &< \left[(1 + 0.05)^{-0} \cdot -3x\right] + \left[(1 + 0.05)^{-1} \cdot 5\right] + \left[(1 + 0.05)^{-2} \cdot x\right] \\
    0 &< \left[1 \cdot -3x\right] + \left[\frac{5}{1 + 0.05}\right] + \left[\frac{x}{(1 + 0.05)^{2}}\right] \\
    0 &< -3x + \frac{5}{1.05} + \frac{x}{1.05^{2}} \\
    0 &< -3 \cdot 1.05^2 x + 5 \cdot 1.05 + x \\
    0 &< -2.3075 x + 5.25 \\
    2.3075 x &< 5.25 \\
    x &< \frac{5.25}{2.3075} \\
    x &\lessapprox 2.275 \\
\end{align}

Therefore, when $r = 5\%$, $P > 0$ holds when $x \lessapprox 2.275$.

We can verify this in code numerically as follows.

In [64]:
r = 0.05
cash_flow = lambda k, x: -3*x if k == 0 else (5 if k == 1 else x)

accept_condition = lambda p: p > 0

# Function to calculate present value based on some value of x
def present_value(x: float) -> float:
    present_value = sum(cash_flow(k, x) * interest.discrete_compound_interest_discounted(r, k, 1) \
                        for k in range(2+1))
    
    return present_value

cash_flows_x: Dict[float, float] = {}
x_min = -100.0
x_max = 100.0
x_step = 0.01
accept_min_x = 0
accept_max_x = None

# Loop over a wide range of x
for x in np.arange(x_min, x_max, x_step):
    cash_flow_x = present_value(x)

    cash_flows_x[x] = cash_flow_x

    if accept_condition(cash_flow_x):
        # Min
        if not accept_min_x:
            accept_min_x = x
        elif x < accept_min_x:
            accept_min_x = x

        # Max
        if not accept_max_x:
            accept_max_x = x
        elif x > accept_max_x:
            accept_max_x = x

if accept_min_x == x_min:
    accept_min_x = 0

if accept_max_x == x_max:
    accept_max_x = math.inf

print(
    f"Range of x such that P > 0 when r = {r * 100}%: {accept_min_x} < x < {accept_max_x}")


Range of x such that P > 0 when r = 5.0%: 0 < x < 2.2700000000523204


## Part B (3 marks)

The IRR (internal rate of return) is $r$ such that $P = 0$. For what range of $x$ will there be a unique, strictly positive IRR?

\begin{align}
    P &= \sum_{k=0}^{2} d(k) \cdot C_k \\
    0 &= \left[d(0) \cdot -3x\right] + \left[d(1) \cdot 5\right] + \left[d(2) \cdot x\right] \\
    &= -3x (1 + r)^{-0} + 5 (1 + r)^{-1} + x (1 + r)^{-2} \\
    &= -3x \cdot 1 + \frac{5}{1 + r} + \frac{x}{(1 + r)^2} \\
    &= -3x (1 + r)^2 + 5 (1 + r) + x \\
    &= -3x (1^2 + 2r + r^2) + 5 + 5r + x \\
    &= -3x - 6xr - 3xr^2 + 5 + 5r + x \\
    &= -3xr^2 + (5 - 6x)r + (x - 3x + 5) \\
    &= -3xr^2 + (5 - 6x)r + (5 - 2x)
\end{align}

As we want $r > 0$, then solve for $\Delta > 0$

\begin{align}
    0 &< \Delta \\
    0 &< (5 - 6x)^2 - 4 \cdot (-3x) \cdot (5 - 2x) \\
    0 &< (25 - 60x + 36x^2) + 12x \cdot (5 - 2x) \\
    0 &< 36x^2 - 60x + 25 + 60x - 24x^2 \\
    0 &< 12x^2 + 25
\end{align}

This will always hold $\forall x \in \mathbb{R}^+$

Therefore, if $x > 0$, then IRR $= \left\{r | P = 0\right\} > 0$.

## Question 3 (8 marks)

| Cashflows ($C_i$) | Times ($t_i$) |
| :---------------: | :-----------: |
| 2.3               | 1.0           |
| 2.9               | 2.0           |
| 3.0               | 3.0           |
| 3.2               | 4.0           |
| 4.0               | 5.0           |
| 3.8               | 6.0           |
| 4.2               | 7.0           |
| 4.8               | 8.0           |
| 5.5               | 9.0           |
| 105               | 10.0          |

<p style="text-align: center;">Table 1: Bond Cashflows</p>

In this question, consider a bond with a set of cashflows given in Table 1. Here, note that the face value $F$ is already included in the last cashflow. Let $y$ be the yield to maturity, $t_i$ be the time of the $i^{\text{th}}$ cashflow $C_i$, and $PV = 100$ be the market price of the bond at $t = 0$. Assume continuous compounding. Then $y$ solves

$$PV = \sum_{i} C_i e^{-y t_i}$$

## Part A (3 marks)

Write out the Newton iteration to compute $y_{n+1}$ from $y_n$ (see L2.49). Specifically, clearly indicate the functions $f(y)$ and $f'(y)$.

## Part B (5 marks)

Implement the above Newton iteration in Matlab using the stopping criteria 

$$|y_{n+1} - y_n|$ < 10^{-8}.$$

Fill in Table 2 for $y_0 = 0.05$ (add rows as necessary).

In addition, try with larger values for $y_0$ and observe the accuracy and convergence speed. How does the performance change?

| $n$ | $y_n$ | $|y_n - y_{n-1}|$ |
| :-: | :---: | :---------------: |
|  0  |  ...  | ..                |
|  0  |  ...  | ..                |
|  0  |  ...  | ..                |
|  0  |  ...  | ..                |
|  0  |  ...  | ..                |

## Question 4

In the Constant Growth DDM model, the present value of the share is")

$$PV = \sum_{t=1}^{\infty} \frac{D_t}{(1 + k)^t}$$

where $D_1, D_2, ...$ are (non-random) dividends and $k > 0$ is the required rate of return

Suppose $D_0 > 0$, $k > 0$ and $g > 0$

Derive the formula for the present value (2) when

$$D_t = D_0 (1 + g)^{\ceil{\frac{t}{2}}}, \hspace{0.5cm} t = 1, 2, ...$$

where $\ceil{x}$ is the smallest integer greater than or equal to $x$. What is the condition of $g$ so that $PV$ is finite? To get full marks, you will need to write an explicit expression (without summation).